In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    data_sets = []
    for filename in filenames:
        data_sets.append(pd.read_csv(os.path.join(dirname, filename)))
sample_sub, training_prompts, test, train = data_sets

In [2]:
train.describe()

,prompt_id,generated
count,1378.000000,1378.000000
mean,0.486212,0.002177
std,0.499991,0.046625
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,1.000000,0.000000
max,1.000000,1.000000


In [3]:
train.dtypes

id           object
prompt_id     int64
text         object
generated     int64
dtype: object

In [4]:
from sklearn.model_selection import train_test_split
import keras_nlp
import keras_core as keras

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend


In [5]:
X = train['text']
y = train['generated']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [6]:
max_words = 10000
padding_length = 200

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Tokenize them into vectors
# Each unique word represented by an integer
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad our data to ensure consistent length
X_train_padded = pad_sequences(X_train_sequences, maxlen=padding_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=padding_length, padding='post', truncating='post')

### Visualizing our preprocessing

Now we can see the three steps our data goes through


In [7]:
essay_number = 5

print("Step 1: Raw Text\n"+ X_train[essay_number])
print("\n\nStep 2: Tokens\n" + str(X_train_sequences[essay_number]))
print("\n\nStep 3: Padded Set\n" + str(X_train_padded[essay_number]))

Step 1: Raw Text
The electrol college system is an unfair system, people don't have the right to select their own president, they dont have the right to select a president. Because, when people vote they are technically voting for the electors for a candied. That candidate can be a democratic or a republican. In source two it states that electors can choose there opposing candidate. Which declines the the whole voting process. Why do we vote? we vote to select a leader who will defend this country and make america a place of opportunity. The most important reason why the electrol cllege is unfair because of the " winners it take all" rule.

We as american people cannot choose their own government, as it says in source two, that when voters vote they are voting for the candidates electors. Each state gets one vote and then the electors can choose who they want for president. In source two it states that the electoral college consists of 538 electors and the most amount of electoral vote